In [1]:
# Cell 1: Import required libraries
import os
import yaml
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import your model and data loading components
from dataloader.dataset_wrapper import create_wrapper_from_dataframe

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# Cell 2: Configuration and paths setup
CONFIG_PATH = "/Users/ivangolov/Desktop/Диплом/CSMP_project/CSMP_thesis_project/services/CSU-MS2/model/qtof_model/median_energy/checkpoints/config.yaml"
TRAIN_CSV_PATH = "/Users/ivangolov/Desktop/Диплом/CSMP_project/CSMP_spectrum_database/data/production/train_deduplicated.csv"
OUTPUT_DIR = "./train_results"

# Device selection for Mac
if torch.backends.mps.is_available():
    DEVICE = 'mps'
    print("Using MPS (Metal Performance Shaders) for GPU acceleration")
elif torch.cuda.is_available():
    DEVICE = 'cuda'
    print("Using CUDA for GPU acceleration")
else:
    DEVICE = 'cpu'
    print("Using CPU")

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Using device: {DEVICE}")
print(f"Validation data path: {TRAIN_CSV_PATH}")

Using MPS (Metal Performance Shaders) for GPU acceleration
Using device: mps
Validation data path: /Users/ivangolov/Desktop/Диплом/CSMP_project/CSMP_spectrum_database/data/production/train_deduplicated.csv


In [3]:
# Cell 3: Load configuration
print("Loading configuration...")
config = yaml.load(open(CONFIG_PATH, "r"), Loader=yaml.FullLoader)

print("Configuration loaded:")
print(f"- Batch size: {config.get('batch_size', 'Not specified')}")
print(f"- Model config keys: {list(config.get('model', {}).keys())}")
print(f"- Loss config: {config.get('loss', {})}")

Loading configuration...
Configuration loaded:
- Batch size: 64
- Model config keys: []
- Loss config: {'temperature': 0.1, 'use_cosine_similarity': True, 'alpha_weight': 0.75}


In [4]:
# Cell 4: Load and explore validation data
print("Loading train data...")
df_train = pd.read_csv(TRAIN_CSV_PATH)

print(f"Validation dataset shape: {df_train.shape}")
print(f"Columns: {list(df_train.columns)}")
print(f"Sample data:")
df_train.head()

Loading train data...
Validation dataset shape: (798444, 10)
Columns: ['peaks_json', 'ion_source', 'compound_source', 'instrument', 'adduct', 'precursor_mz', 'smiles', 'inchikey', 'ion_mode', 'molecular_formula']
Sample data:


,peaks_json,ion_source,compound_source,instrument,adduct,precursor_mz,smiles,inchikey,ion_mode,molecular_formula
0,"[[42.014248, 0.10199999999999998], [42.26601, ...",ESI,Crude,Orbitrap,[M+H]+,377.186,CC12CCC(C(=O)N(CNc3cc4c(cc3)c3ccccc3o4)C1=O)C2...,RNKMIWQDRWSWCD-UHFFFAOYSA-N,Positive,C23H24N2O3
1,"[[49.01717, 0.155], [49.020023, 0.253], [67.05...",ESI,Crude,Orbitrap,[M+H]+,377.186,CC12CCC(C(=O)N(CNc3cc4c(cc3)c3ccccc3o4)C1=O)C2...,RNKMIWQDRWSWCD-UHFFFAOYSA-N,Positive,C23H24N2O3
2,"[[49.017338, 0.242], [49.020237, 0.181], [67.0...",ESI,Crude,Orbitrap,[M+H]+,377.186,CC12CCC(C(=O)N(CNc3cc4c(cc3)c3ccccc3o4)C1=O)C2...,RNKMIWQDRWSWCD-UHFFFAOYSA-N,Positive,C23H24N2O3
3,"[[49.01701, 0.144], [49.019947, 0.244], [139.0...",ESI,Crude,Orbitrap,[M+H]+,377.186,CC12CCC(C(=O)N(CNc3cc4c(cc3)c3ccccc3o4)C1=O)C2...,RNKMIWQDRWSWCD-UHFFFAOYSA-N,Positive,C23H24N2O3
4,"[[49.017166, 0.155], [49.020008, 0.253], [139....",ESI,Crude,Orbitrap,[M+H]+,377.186,CC12CCC(C(=O)N(CNc3cc4c(cc3)c3ccccc3o4)C1=O)C2...,RNKMIWQDRWSWCD-UHFFFAOYSA-N,Positive,C23H24N2O3


In [5]:
df_train_sample = df_train.sample(n=10000,random_state=42).reset_index(drop=True)

In [7]:
# Cell 6: Prepare validation data loader
print("Preparing data loaders")

# Create data wrapper from DataFrame
wrapper, processed_df = create_wrapper_from_dataframe(
    df=df_train_sample,
    batch_size=64,  
    num_workers=8,
    valid_size=0.2,  
    use_ddp=False,
    output_dir="./train_features",
)

# Get the data loader
train_loader, val_loader = wrapper.get_data_loaders()

print(f"Successfully processed {len(processed_df)} samples")
print(f"Number of batches: {len(val_loader)}")

Preparing data loaders
Convert DataFrame to compatible files
Processed 10000 valid spectra out of 10000 total entries.
Create data wrapper
calculating molecular graphs


 13%|█▎        | 1033/8000 [00:00<00:04, 1515.42it/s]

SMILES [I-].O=C(OCC1=CC[N+]2(C)CCC(O)C12)C(O)(C(O)C)C(C)C calculation failure


 22%|██▏       | 1738/8000 [00:01<00:03, 1708.82it/s]

SMILES [Cl-].O=C1C2=CC=C(O)C(=C2OC(=C1C=3C=CC=4OCCCOC4C3)C)C[NH+](C)C calculation failure
SMILES [Na+].O=C(CCCCCCCCCCC)CC(O)S(=O)(=O)[O-] calculation failure


...

SMILES [Na+].O=C(CCCCCCCCCCC)CC(O)S(=O)(=O)[O-] calculation failure
SMILES [K+].O=S(=O)([O-])ON=C(SC1OC(CO)C(O)C(O)C1O)CC=C.O calculation failure


 92%|█████████▏| 1837/2000 [00:01<00:00, 1679.95it/s]

SMILES [Br-].O=C(OC1CC2C3OC3C(C1)[N+]2(C)CCCC)C(C=4C=CC=CC4)CO calculation failure
SMILES CCC1=C(C2=NC1=CC3=C(C4=C([N-]3)C(=C5[C@H]([C@@H](C(=N5)C=C6C(=C(C(=C2)[N-]6)C=C)C)C)CCC(=O)OC/C=C(\C)/CCC[C@H](C)CCC[C@H](C)CCCC(C)C)[C@H](C4=O)C(=O)OC)C)C=O.[Mg+2] calculation failure
SMILES [I-].O=C(OCC1=CC[N+]2(C)CCC(O)C12)C(O)(C(O)C)C(C)C calculation failure


100%|██████████| 2000/2000 [00:01<00:00, 1644.40it/s]

Calculated 1705 molecular graph-mass spectrometry pairs
Successfully processed 10000 samples
Number of batches: 27


In [8]:
from model import ModelCLR

# Initialize model architecture
model = ModelCLR(**config["model_config"]).to(DEVICE)

In [9]:
# Model Architecture Overview
print("MODEL ARCHITECTURE:")
print("-" * 40)
print(f"Model Type: {type(model).__name__}")
print(f"Device: {DEVICE}")

MODEL ARCHITECTURE:
----------------------------------------
Model Type: ModelCLR
Device: mps


In [10]:
# Submodules Analysis
print("SUBMODULES:")
print("-" * 40)
for name, module in model.named_children():
    print(f"{name:15}: {type(module).__name__}")
    if hasattr(module, '__dict__'):
        for attr_name, attr_value in module.__dict__.items():
            if not attr_name.startswith('_') and not callable(attr_value):
                if isinstance(attr_value, (int, float, str, bool)):
                    print(f"  {attr_name:12}: {attr_value}")

SUBMODULES:
----------------------------------------
Smiles_model   : SmilesModel
  training    : True
  num_layer   : 5
  emb_dim     : 300
  feat_dim    : 512
  drop_ratio  : 0.3
MS_model       : MSModel
  training    : True
smi_esa        : ESA_SMILES
  training    : True
spec_esa       : ESA_SPEC
  training    : True
smi_proj       : Linear
  training    : True
  in_features : 256
  out_features: 256
spec_proj      : Linear
  training    : True
  in_features : 256
  out_features: 256


In [11]:
# Parameter Count
print("PARAMETER ANALYSIS:")
print("-" * 40)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")
print(f"Non-trainable Parameters: {total_params - trainable_params:,}")


PARAMETER ANALYSIS:
----------------------------------------
Total Parameters: 5,318,572
Trainable Parameters: 5,318,444
Non-trainable Parameters: 128


In [12]:
# Memory Usage (approximate)
print("MEMORY ANALYSIS:")
print("-" * 40)
param_size = sum(p.numel() * p.element_size() for p in model.parameters())
buffer_size = sum(b.numel() * b.element_size() for b in model.buffers())
model_size_mb = (param_size + buffer_size) / 1024 / 1024

print(f"Model Size: {model_size_mb:.2f} MB")
print(f"Parameter Memory: {param_size / 1024 / 1024:.2f} MB")
print(f"Buffer Memory: {buffer_size / 1024 / 1024:.2f} MB")


MEMORY ANALYSIS:
----------------------------------------
Model Size: 20.30 MB
Parameter Memory: 20.29 MB
Buffer Memory: 0.01 MB


In [13]:
from loss.nt_xent import NTXentLoss

# Initialize loss function
temperature = config.get('loss', {}).get('temperature', 0.1)
batch_size = config.get('batch_size', 1024)
use_cosine_similarity = config.get('loss', {}).get('use_cosine_similarity', True)
alpha_weight = config.get('loss', {}).get('alpha_weight', 1.0)

criterion = NTXentLoss(
    device=DEVICE, 
    batch_size=batch_size, 
    temperature=temperature, 
    use_cosine_similarity=use_cosine_similarity, 
    alpha_weight=alpha_weight
)

In [14]:
# Cell 15: Training Setup and Optimizer
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import json
import time
from collections import defaultdict

print("Setting up training components...")
OUTPUT_DIR = "./model"

# Initialize optimizer
optimizer = optim.AdamW(
    model.parameters(),
    lr=float(config.get('learning_rate', 5e-6)),
    weight_decay=float(config.get('weight_decay', 1e-4))
)

# Initialize scheduler
scheduler = CosineAnnealingLR(
    optimizer, 
    T_max=config.get('epochs', 4000),
    eta_min=1e-7
)

# Training configuration
epochs = config.get('epochs', 4000)
eval_every_n_epochs = config.get('eval_every_n_epochs', 5)
log_every_n_steps = config.get('log_every_n_steps', 2)

# Create checkpoint directory
checkpoint_dir = os.path.join(OUTPUT_DIR, "checkpoints")
os.makedirs(checkpoint_dir, exist_ok=True)

print(f"Training for {epochs} epochs")
print(f"Optimizer: AdamW with lr={config.get('learning_rate', 5e-6)}")
print(f"Scheduler: CosineAnnealingLR")
print(f"Checkpoint directory: {checkpoint_dir}")

Setting up training components...
Training for 100 epochs
Optimizer: AdamW with lr=5e-06
Scheduler: CosineAnnealingLR
Checkpoint directory: ./model/checkpoints


In [15]:
# Cell 16: Training and Evaluation Functions
def train_epoch(model, train_loader, criterion, optimizer, device, epoch):
    model.train()
    total_loss = 0
    num_batches = 0
    batch_losses = []
    
    progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch}")
    
    for batch_idx, (graphs, mzs, intensities, num_peaks) in enumerate(progress_bar):
        # Move data to device
        graphs = graphs.to(device)
        mzs = mzs.to(device)
        intensities = intensities.to(device)
        num_peaks = num_peaks.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        mol_features, spec_features = model(graphs, mzs, intensities, num_peaks)
        loss = criterion(mol_features, spec_features)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Track loss
        batch_loss = loss.item()
        total_loss += batch_loss
        batch_losses.append(batch_loss)
        num_batches += 1
        
        # Update progress bar
        progress_bar.set_postfix({
            'Loss': f'{batch_loss:.4f}',
            'Avg Loss': f'{total_loss/num_batches:.4f}'
        })
        
        # Log every n steps
        if batch_idx % log_every_n_steps == 0:
            print(f"Epoch {epoch}, Batch {batch_idx}, Loss: {batch_loss:.4f}")
    
    avg_loss = total_loss / num_batches
    return avg_loss, batch_losses

In [16]:
def compute_retrieval_metrics(molecular_features, spectral_features):
    from sklearn.metrics import pairwise_distances

    # Compute cosine distance matrix
    distance_matrix = pairwise_distances(molecular_features, spectral_features, metric='cosine')
    
    # For each molecular feature, find the rank of the correct spectral feature
    ranks = []
    for i in range(distance_matrix.shape[0]):
        sorted_indices = np.argsort(distance_matrix[i])
        rank = np.where(sorted_indices == i)[0][0] + 1  # +1 for 1-based rank
        ranks.append(rank)
    
    ranks = np.array(ranks)
    
    # Compute top-k accuracies
    top_1_accuracy = np.mean(ranks <= 1)
    top_5_accuracy = np.mean(ranks <= 5)
    top_10_accuracy = np.mean(ranks <= 10)
    
    return {
        'top_1_accuracy': top_1_accuracy,
        'top_5_accuracy': top_5_accuracy,
        'top_10_accuracy': top_10_accuracy
    }


def evaluate_model(model, val_loader, criterion, device, epoch):
    model.eval()
    total_loss = 0
    num_batches = 0
    
    molecular_features_list = []
    spectral_features_list = []
    
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc=f"Evaluating Epoch {epoch}")
        
        for batch_idx, (graphs, mzs, intensities, num_peaks) in enumerate(progress_bar):
            # Move data to device
            graphs = graphs.to(device)
            mzs = mzs.to(device)
            intensities = intensities.to(device)
            num_peaks = num_peaks.to(device)
            
            # Forward pass
            mol_features, spec_features = model(graphs, mzs, intensities, num_peaks)
            loss = criterion(mol_features, spec_features)
            
            total_loss += loss.item()
            num_batches += 1
            
            # Store features for retrieval metrics
            molecular_features_list.append(mol_features.cpu().numpy())
            spectral_features_list.append(spec_features.cpu().numpy())
            
            progress_bar.set_postfix({'Val Loss': f'{loss.item():.4f}'})
    
    avg_loss = total_loss / num_batches
    
    # Compute retrieval metrics
    all_mol_features = np.vstack(molecular_features_list)
    all_spec_features = np.vstack(spectral_features_list)
    
    # Compute cosine similarities
    cosine_similarities = np.sum(all_mol_features * all_spec_features, axis=1)
    mean_similarity = np.mean(cosine_similarities)
    
    # Compute retrieval metrics
    retrieval_metrics = compute_retrieval_metrics(all_mol_features, all_spec_features)
    
    return avg_loss, mean_similarity, retrieval_metrics

In [17]:
def save_checkpoint(model, optimizer, scheduler, epoch, train_loss, val_loss, val_metrics, checkpoint_dir):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
        'val_metrics': val_metrics,
        'config': config
    }
    
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch}.pth')
    torch.save(checkpoint, checkpoint_path)
    print(f"Checkpoint saved: {checkpoint_path}")
    
    # Save best model
    best_checkpoint_path = os.path.join(checkpoint_dir, 'best_model.pth')
    if not os.path.exists(best_checkpoint_path):
        torch.save(checkpoint, best_checkpoint_path)
        print(f"Best model saved: {best_checkpoint_path}")
    else:
        best_checkpoint = torch.load(best_checkpoint_path)
        if val_loss < best_checkpoint['val_loss']:
            torch.save(checkpoint, best_checkpoint_path)
            print(f"New best model saved: {best_checkpoint_path}")

print("Training and evaluation functions defined!")

Training and evaluation functions defined!


In [18]:
# Cell 17: Training Loop
# Training history for plotting
train_history = {
    'epochs': [],
    'train_losses': [],
    'val_losses': [],
    'val_similarities': [],
    'val_top1_acc': [],
    'val_top5_acc': [],
    'val_top10_acc': [],
    'learning_rates': []
}

best_val_loss = float('inf')
start_time = time.time()

print("Starting training...")
print("=" * 60)

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    
    # Training phase
    train_loss, batch_losses = train_epoch(
        model, train_loader, criterion, optimizer, DEVICE, epoch
    )
    
    # Validation phase (every n epochs)
    if epoch % eval_every_n_epochs == 0:
        val_loss, val_similarity, val_metrics = evaluate_model(
            model, val_loader, criterion, DEVICE, epoch
        )
        
        # Update learning rate
        scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']
        
        # Log results
        epoch_time = time.time() - epoch_start_time
        total_time = time.time() - start_time
        
        print(f"\nEpoch {epoch}/{epochs}")
        print(f"Train Loss: {train_loss:.6f}")
        print(f"Val Loss: {val_loss:.6f}")
        print(f"Val Similarity: {val_similarity:.4f}")
        print(f"Top-1 Accuracy: {val_metrics['top_1_accuracy']:.4f}")
        print(f"Top-5 Accuracy: {val_metrics['top_5_accuracy']:.4f}")
        print(f"Top-10 Accuracy: {val_metrics['top_10_accuracy']:.4f}")
        print(f"Learning Rate: {current_lr:.2e}")
        print(f"Epoch Time: {epoch_time:.2f}s, Total Time: {total_time:.2f}s")
        print("-" * 60)
        
        # Store history
        train_history['epochs'].append(epoch)
        train_history['train_losses'].append(train_loss)
        train_history['val_losses'].append(val_loss)
        train_history['val_similarities'].append(val_similarity)
        train_history['val_top1_acc'].append(val_metrics['top_1_accuracy'])
        train_history['val_top5_acc'].append(val_metrics['top_5_accuracy'])
        train_history['val_top10_acc'].append(val_metrics['top_10_accuracy'])
        train_history['learning_rates'].append(current_lr)
        
        # Save checkpoint
        save_checkpoint(
            model, optimizer, scheduler, epoch, 
            train_loss, val_loss, val_metrics, checkpoint_dir
        )
        
        # Save training history
        history_path = os.path.join(OUTPUT_DIR, 'training_history.json')
        with open(history_path, 'w') as f:
            json.dump(train_history, f, indent=2)
    
    else:
        # Just update learning rate and log basic info
        scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']
        epoch_time = time.time() - epoch_start_time
        
        print(f"Epoch {epoch}/{epochs} - Train Loss: {train_loss:.6f}, LR: {current_lr:.2e}, Time: {epoch_time:.2f}s")

print("\nTraining completed!")
print(f"Total training time: {(time.time() - start_time)/3600:.2f} hours")

Starting training...


Training Epoch 1:   1%|          | 1/107 [00:21<37:11, 21.05s/it, Loss=4.1717, Avg Loss=4.1717]

Epoch 1, Batch 0, Loss: 4.1717


Training Epoch 1:   3%|▎         | 3/107 [00:22<09:15,  5.34s/it, Loss=4.1893, Avg Loss=4.1826]

Epoch 1, Batch 2, Loss: 4.1893


...

Epoch 1/100 - Train Loss: 4.161294, LR: 5.00e-06, Time: 170.10s


Training Epoch 2:   1%|          | 1/107 [00:25<45:37, 25.82s/it, Loss=4.1535, Avg Loss=4.1535]

Epoch 2, Batch 0, Loss: 4.1535


libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x118f0c430>
Traceback (most recent call last):
  File "/Users/ivangolov/anaconda3/envs/CSU-MS2/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1476, in __del__
   

: 